In [1]:
sentences = [
    "I want to book a fly",
    "I need to book a hotel",
    "I want to book a car",
    "I read a lot of books last year",
    "This book is amazing",
    "The old man and the see"
]

In [22]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

In [24]:
# Use a service account.
cred = credentials.Certificate("goblob-95e2a-6add9b68fd5d.json")
# cred = credentials.Certificate("aiuda-ffc77-b8546e446401.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [25]:
profile = db.collection("profiles")
docs = profile.get()

In [26]:
df_profiles = []

for doc in docs:
    user_profile = doc.to_dict()

    if user_profile["location"] is not None:
        try:
            location = {
                "lat": user_profile["location"].latitude,
                "long": user_profile["location"].longitude,
            }
        except:
            location = {
                "lat": 0,
                "long": 0,
            }
    else:
        location = {
            "lat": 0,
            "long": 0,
        }

    tags_array = []

    for tag in doc.to_dict()["tags"]:
        tags_array.append(tag["text"])

    data = {
        "name": user_profile["displayName"],
        "address": user_profile["address"] if "address" in user_profile.keys() is not None else "",
        "gender": user_profile["gender"],
        "age": user_profile["age"],
        # rating = user_profile["avgRating"]
        "tags": tags_array,
        "service": user_profile["service"]["text"],
        "about": user_profile["about"],
        "location": location
    }

    df_profiles.append(data)

df_profiles = pd.DataFrame.from_dict(df_profiles)

df_profiles

,name,address,gender,age,tags,service,about,location
0,Violeta IOS,"Calle Punta Colón, Calle Punta Colón, Panama C...",female,29,"[pets, paseo_de_perros, dog_sitting]",pets,Guardo y paseo de mascotas. Soy estudiante 3er...,"{'lat': 8.97548104431072, 'long': -79.50703719..."
1,violeta ios 0.7.17,,female,35,"[pets, dog_sitting, paseo_de_perros]",pets,"¡Hola! Soy Violeta, una apasionada por los ani...","{'lat': 8.9837843, 'long': -79.5207418}"
2,Arizza,"Panamá, , Panamá, Panama",,None,[user],user,,"{'lat': 8.994300842285156, 'long': -79.5596818..."
3,Oriana Vega24422,"PQ5V+3PR, PQ5V+3PR, Punto Fijo, Falcón, Venezuela",female,27,"[user, musica, piano, moda]",user,,"{'lat': 11.7080935, 'long': -70.2067349}"
4,Alejandro,"Panamá, P.H. Bay View, Panamá, Provincia de Pa...",male,32,"[misc, marketing]",misc,,"{'lat': 8.9600785, 'long': -79.5383008}"
...,...,...,...,...,...,...,...,...
69,Angela,"Panamá, El Mare Bethania, Panamá, Provincia de...",female,33,"[spa, masajes]",spa,,"{'lat': 8.9946661, 'long': -79.5328797}"
70,Laissa,"Panamá, , Panamá, Panama",,,"[transport, uber, moto]",transport,"Soy Laissa, una apasionada del transporte y la...","{'lat': 8.9767517, 'long': -79.5326233}"
71,Janicy dos Santos,"55-7, 75 a Oeste, Panamá, Provincia de Panamá,...",male,32,"[misc, educacion, tutorias, psicopedagoga]",misc,,"{'lat': 9.0157276, 'long': -79.5295287}"
72,HomeSecurePTY,"Homesecurepty, Calle 66 Este, Panamá, Provinci...",male,44,"[repair, cerrajeria, cerrajeria_automotriz, ll...",repair,"Cerrajería Automotriz, Residencial Y Comercial.","{'lat': 8.995956399999997, 'long': -79.5131565}"


In [27]:
df_profiles = df_profiles.dropna()
df_profiles = df_profiles[~df_profiles['age'].isin([''])]
df_profiles['age'] = df_profiles.age.astype(int)
df_profiles

,name,address,gender,age,tags,service,about,location
0,Violeta IOS,"Calle Punta Colón, Calle Punta Colón, Panama C...",female,29,"[pets, paseo_de_perros, dog_sitting]",pets,Guardo y paseo de mascotas. Soy estudiante 3er...,"{'lat': 8.97548104431072, 'long': -79.50703719..."
1,violeta ios 0.7.17,,female,35,"[pets, dog_sitting, paseo_de_perros]",pets,"¡Hola! Soy Violeta, una apasionada por los ani...","{'lat': 8.9837843, 'long': -79.5207418}"
3,Oriana Vega24422,"PQ5V+3PR, PQ5V+3PR, Punto Fijo, Falcón, Venezuela",female,27,"[user, musica, piano, moda]",user,,"{'lat': 11.7080935, 'long': -70.2067349}"
4,Alejandro,"Panamá, P.H. Bay View, Panamá, Provincia de Pa...",male,32,"[misc, marketing]",misc,,"{'lat': 8.9600785, 'long': -79.5383008}"
5,Jules D.,"32, Rue Fabert, Paris, Île-de-France, France",male,28,[user],user,,"{'lat': 48.8598247, 'long': 2.3113472}"
...,...,...,...,...,...,...,...,...
68,Tarandeep singh,"MHRG+CPG, Old Bazaar Road, Ratia, Haryana, India",male,26,[repair],repair,,"{'lat': 29.6916441, 'long': 75.5780667}"
69,Angela,"Panamá, El Mare Bethania, Panamá, Provincia de...",female,33,"[spa, masajes]",spa,,"{'lat': 8.9946661, 'long': -79.5328797}"
71,Janicy dos Santos,"55-7, 75 a Oeste, Panamá, Provincia de Panamá,...",male,32,"[misc, educacion, tutorias, psicopedagoga]",misc,,"{'lat': 9.0157276, 'long': -79.5295287}"
72,HomeSecurePTY,"Homesecurepty, Calle 66 Este, Panamá, Provinci...",male,44,"[repair, cerrajeria, cerrajeria_automotriz, ll...",repair,"Cerrajería Automotriz, Residencial Y Comercial.","{'lat': 8.995956399999997, 'long': -79.5131565}"


In [32]:
sentences = df_profiles[df_profiles["about"] != ""].about.values

In [37]:
services = df_profiles[df_profiles["about"] != ""].service.values

In [13]:
import pandas as pd

from langchain_openai import OpenAIEmbeddings

In [64]:
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import seaborn as sns

# Sentence data as a list
# sentences = data['text'].tolist()

# Sentences to embeddings
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5")
# embedding_model = SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")
sentence_embeddings = embedding_model.encode(sentences)

# Reduce embedding dimensionality using t-SNE for visualization
X_embedded = TSNE(n_components=2, perplexity=5, n_iter=10000).fit_transform(
    sentence_embeddings
)

/Users/nmlemus/miniforge3/envs/yomap/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



In [ ]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0: "x", 1: "y"})
df_embeddings = df_embeddings.assign(label=services)

df_embeddings = df_embeddings.assign(text=sentences)

In [41]:
import plotly.express as px

fig = px.scatter(
    df_embeddings,
    x="x",
    y="y",
    color="label",
    #labels={"color": "label"},
    hover_data=["text"],
    #title="SurePath Data Augmentation Embedding Visualization",
)
fig.show()

# Synthetic Dataset

In [42]:
import json

In [46]:
def load_dataset(name):

    # Load JSON data from a file
    with open(name, "r") as file:
        data = json.load(file)

    training_chatgpt = pd.DataFrame.from_dict(data)

    dataset = []

    # Customizations to match the required format
    for idx, row in training_chatgpt.iterrows():
        for about_me_examples in row.about_me_examples:
            data = {
                "intent": row.category,
                "sentence": about_me_examples,
            }

            dataset.append(data)

    dataset = pd.DataFrame.from_dict(dataset)
    dataset = dataset.sort_values("intent")
    dataset["label"] = dataset["intent"]

    # Convert Labels to Int
    dataset["label"] = pd.Categorical(dataset["label"])
    dataset["label"] = dataset["label"].astype("category").cat.codes

    return dataset


dataset = load_dataset("/Users/nmlemus/projects/yomap/topic_modeling/about_me_examples.json")

In [47]:
dataset

,intent,sentence,label
24,8_56_hierro,Need reliable ironwork? I provide professional...,0
20,8_56_hierro,I design and fabricate custom ironwork solutio...,0
21,8_56_hierro,"Providing expert ironwork services, including ...",0
23,8_56_hierro,"Offering comprehensive ironwork services, from...",0
22,8_56_hierro,"With years of experience in iron fabrication, ...",0
...,...,...,...
208,vegetariano,"Dedicated to promoting sustainable eating, I o...",41
205,vegetariano,I specialize in crafting delicious and nutriti...,41
206,vegetariano,"Providing vegetarian catering services, I crea...",41
207,vegetariano,"As a personal chef, I prepare balanced vegetar...",41


In [62]:
df = []
questions = {
    "intent": "question",
    "sentence": "I need to fix my car lock"
}

df.append(questions)

df = pd.DataFrame.from_dict(df)

dataset = pd.concat([dataset, df])

dataset

,intent,sentence,label
24,8_56_hierro,Need reliable ironwork? I provide professional...,0.0
20,8_56_hierro,I design and fabricate custom ironwork solutio...,0.0
21,8_56_hierro,"Providing expert ironwork services, including ...",0.0
23,8_56_hierro,"Offering comprehensive ironwork services, from...",0.0
22,8_56_hierro,"With years of experience in iron fabrication, ...",0.0
...,...,...,...
207,vegetariano,"As a personal chef, I prepare balanced vegetar...",41.0
209,vegetariano,"With a passion for plant-based cooking, I brin...",41.0
0,question,Busco a alguien que me arregle un pantalon,NaN
0,question,Necesito arreglar la cerradura de mi auto,NaN


In [63]:
sentences = dataset.sentence.values

In [65]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0: "x", 1: "y"})
df_embeddings = df_embeddings.assign(label=dataset.intent.values)

df_embeddings = df_embeddings.assign(text=sentences)

In [66]:
fig = px.scatter(
    df_embeddings,
    x="x",
    y="y",
    color="label",
    #labels={"color": "label"},
    hover_data=["text"],
    #title="SurePath Data Augmentation Embedding Visualization",
)
fig.show()